In [25]:
import pandas as pd
import datetime

In [26]:
ruta=r"C:\Users\jcleiva\OneDrive - Grupo-exito.com\Escritorio\Proyectos\Reportes Base"
rutaM=r"C:\Users\jcleiva\OneDrive - Grupo-exito.com\Escritorio\Proyectos\Reportes Base\Maestras"
rutaR=r"C:\Users\jcleiva\OneDrive - Grupo-exito.com\Escritorio\Proyectos\Reportes"

In [27]:
def month_year_iter(start_month, start_year, end_month, end_year):
    ym_start= 12*start_year + start_month - 1
    ym_end= 12*end_year + end_month - 1
    for ym in range(ym_start, ym_end):
        y, m = divmod(ym, 12)
        yield y, m+1

In [28]:
colsComp=["Orden","Centro","Almacén","Material","Texto breve material",
          "Unidad medida base (=EINHEIT)","Cantidad necesaria (EINHEIT)",
          "Cantidad tomada (EINHEIT)","Valor de la toma (WAERS)","Clase de movimiento",
          "Precio/MonL (WAERS)"]

convComp={"Orden":str,"Centro":str,"Almacén":str,"Pos.lista componentes":str,"Material":str,
          "Clase de movimiento":str}

In [29]:
colsCab=["Centro","Orden","Número material","Texto breve material","Clase de orden",
         "Planificador nec.","Resp.ctrl.producción","Cantidad orden (GMEIN)","Cantidad entregada (GMEIN)",
         "Unidad de medida (=GMEIN)","Status de sistema","Cantidad notificada (GMEIN)",
         "Ctd.confirmada (GMEIN)","Versión fabricación","Cantidad notificada (CONF_REM_UNIT)",
         "Difer.confirm.proc.","División","Fecha liberac.real"]

convCab={"Centro":str,"Orden":str,"Número material":str,"Difer.confirm.proc.":str,"División":str}

In [30]:
colsAdi=["Orden","Centro","Almacén","Material","Texto de material",
         "Unidad medida base (=MEINS)","Ctd.en UM base (MEINS)","Importe ML (WAERS)",
         "Clase de movimiento"]

convAdi={"Orden":str,"Centro":str,"Almacén":str,"Pos.documento mat.":str,"Material":str,
         "Clase de movimiento":str}

colsMb51=["Centro","Orden","Material","Ctd.en UM entrada","Importe ML","Clase de movimiento"]
convMb51={"Centro":str,"Orden":str,"Material":str,"Clase de movimiento":str}

In [31]:
colsEjec=["Clase de coste","Denom.clase de coste","Orden partner","Valor/mon.inf.","Cantidad total","Desv.precio fija"]
convEjec={"Clase de coste":str,"Orden partner":str}

In [32]:
mesA=1
yearA=2023
mesI=6
yearI=2023

In [33]:
for tiempo in month_year_iter(mesA,yearA,mesI,yearI):
    dfComp=pd.read_excel(ruta+"\{}\Consumos\{}. Componentes.xlsx".format(tiempo[0],tiempo[1]),usecols=colsComp,converters=convComp)
    dfAdi=pd.read_excel(ruta+"\{}\Consumos\{}. Adicionales.xlsx".format(tiempo[0],tiempo[1]),usecols=colsAdi,converters=convAdi)
    dfCab=pd.read_excel(ruta+"\{}\Consumos\{}. Cabeceras de orden.xlsx".format(tiempo[0],tiempo[1]),usecols=colsCab,converters=convCab)
    
    dfAdi=dfAdi.rename(columns={"Unidad medida base (=MEINS)":"Unidad medida base (=EINHEIT)",
                     'Ctd.en UM base (MEINS)':'Cantidad tomada (EINHEIT)',
                     "Importe ML (WAERS)":'Valor de la toma (WAERS)',
                    "Texto de material":"Texto breve material"})

    dfAdi["Precio/MonL (WAERS)"]=dfAdi["Valor de la toma (WAERS)"].divide(dfAdi["Cantidad tomada (EINHEIT)"],fill_value=0)

    dfComp=pd.concat([dfComp,dfAdi],sort=False)
    del dfAdi

    dfComp["Almacén"].fillna("No encontrado",inplace=True)
    dfComp["Cantidad necesaria (EINHEIT)"].fillna(0,inplace=True)

    dfCab["Status"]=dfCab["Status de sistema"].str[:4]

    dfCab=dfCab.rename(columns={"Cantidad orden (GMEIN)":"Cantidad Plan PT","Cantidad entregada (GMEIN)":"Cantidad Real PT",
                               'Número material':"Receta",'Texto breve material':"Desc. Receta"})

    dfCab=dfCab[['Centro', 'Orden', 'Receta', 'Desc. Receta', 'Cantidad Plan PT',
           'Cantidad Real PT', 'División', 'Fecha liberac.real', 'Status']]
    
    dfCab["División"].fillna("No encontrado",inplace=True)

    if not dfComp.shape[0]==dfComp.merge(dfCab,how="left",on=['Centro', 'Orden']).shape[0]:
        raise Exception("Filas añadidas")

    dfComp=dfComp.merge(dfCab,how="left",on=['Centro', 'Orden'])

    dfComp["Cantidad Plan MP"]=dfComp["Cantidad Real PT"].divide(dfComp["Cantidad Plan PT"],fill_value=0)*dfComp["Cantidad necesaria (EINHEIT)"]
    dfComp=dfComp.rename(columns={"Precio/MonL (WAERS)":"Precio Plan"})
    
    dfM=pd.read_excel(rutaM+"\MM60.xlsx",usecols=["Material","Centro","Cantidad base","Precio","Control de precios"],
                      converters={"Material":str,"Centro":str})
    dfM=dfM.drop_duplicates(subset=["Material","Centro"])
    if not dfComp.shape[0]==dfComp.merge(dfM,on=["Centro","Material"],how="left").shape[0]:
        raise Exception("MM60 inserta datos")
        
    dfM=dfM.rename(columns={"Precio":"Precio MM60"})
    dfM["Precio MM60"]=dfM["Precio MM60"].divide(dfM["Cantidad base"],fill_value=0).fillna(0)
    
    dfComp=dfComp.merge(dfM,on=["Centro","Material"],how="left")
    #del dfM
    dfComp["Cantidad base"].fillna(1,inplace=True)
    dfComp["Precio Plan"]=dfComp["Precio Plan"].divide(dfComp["Cantidad base"])
    
    dfComp["Precio tomado"]=dfComp["Valor de la toma (WAERS)"].divide(dfComp["Cantidad tomada (EINHEIT)"],fill_value=0)
    dfComp["Precio tomado"].fillna(0,inplace=True)
    dfComp["Precio esperado"] = dfComp["Precio Plan"]
    dfComp.loc[dfComp["Precio Plan"]==0.0,"Precio esperado"]=dfComp.loc[dfComp["Precio Plan"]==0.0,"Precio tomado"]
    
    dfMb51=pd.read_excel(ruta+"\{}\Consumos\{}. MB51 (Consumos).xlsx".format(tiempo[0],tiempo[1]),usecols=colsMb51,converters=convMb51)

    dfMb51["Clase de movimiento"]=dfMb51["Clase de movimiento"].replace(["262","102","532"],["261","101","531"])

    dfMb51=dfMb51.groupby(["Orden","Centro","Material","Clase de movimiento"]).sum().reset_index()

    dfMb51["Precio MB51"]=dfMb51["Importe ML"].divide(dfMb51["Ctd.en UM entrada"],fill_value=0).fillna(0)

    if not dfComp.shape[0]==dfComp.merge(dfMb51[["Orden","Centro","Material","Clase de movimiento","Precio MB51"]],
                                  on=["Orden","Centro","Material","Clase de movimiento"],
                                 how="left").shape[0]:
        raise Exception("Mb51 añade valores")

    dfComp=dfComp.merge(dfMb51[["Orden","Centro","Material","Clase de movimiento","Precio MB51"]],
                                  on=["Orden","Centro","Material","Clase de movimiento"],
                                 how="left")
    
    dfComp["Precio MB51"].fillna(0,inplace=True)
    dfComp["Precio MM60"].fillna(0,inplace=True)
    
    dfComp.loc[dfComp["Precio esperado"]==0.0,"Precio esperado"]=dfComp.loc[dfComp["Precio esperado"]==0.0,"Precio MB51"]
    dfComp.loc[dfComp["Precio Plan"]==0.0,"Precio Plan"]=dfComp.loc[dfComp["Precio Plan"]==0.0,"Precio MB51"]
    
    dfComp.loc[dfComp["Precio esperado"]==0.0,"Precio esperado"]=dfComp.loc[dfComp["Precio esperado"]==0.0,"Precio MM60"]
    dfComp.loc[dfComp["Precio Plan"]==0.0,"Precio Plan"]=dfComp.loc[dfComp["Precio Plan"]==0.0,"Precio MM60"]

    dfComp.loc[dfComp["Precio tomado"]==0.0,"Precio tomado"]=dfComp.loc[dfComp["Precio tomado"]==0.0,"Precio MB51"]    
    dfComp.loc[dfComp["Precio tomado"]==0.0,"Precio tomado"]=dfComp.loc[dfComp["Precio tomado"]==0.0,"Precio MM60"]
    
    dfComp["Costo Tomado"]=dfComp["Cantidad tomada (EINHEIT)"]*dfComp["Precio tomado"]
    dfComp.loc[dfComp["Costo Tomado"]==0.0,"Costo Tomado"]=dfComp.loc[dfComp["Costo Tomado"]==0.0,"Valor de la toma (WAERS)"]
    dfComp.loc[dfComp["Valor de la toma (WAERS)"]==0.0,"Valor de la toma (WAERS)"]=dfComp.loc[dfComp["Valor de la toma (WAERS)"]==0.0,"Costo Tomado"]
    
    dfComp["Costo Estándar"]=dfComp["Cantidad Plan MP"]*dfComp["Precio Plan"]
    dfComp["Costo Esperado"]=dfComp["Cantidad tomada (EINHEIT)"]*dfComp["Precio esperado"]

    dfComp["Variación Consumo"]=dfComp["Costo Esperado"]-dfComp["Costo Estándar"]
    dfComp["Variación Precio"]=dfComp["Costo Tomado"]-dfComp["Costo Esperado"]
    del dfComp['Cantidad base']
    dfComp=dfComp[['Orden', 'Centro', 'Receta', 'Desc. Receta','Clase de movimiento', 'Material', 'Texto breve material',
           'Unidad medida base (=EINHEIT)', 'Cantidad necesaria (EINHEIT)','Cantidad tomada (EINHEIT)',
            'Valor de la toma (WAERS)', 'Almacén','División','Fecha liberac.real', 'Status',
           'Cantidad Plan PT', 'Cantidad Real PT', 
            'Cantidad Plan MP', 'Precio tomado', 'Precio esperado', 'Precio Plan',"Precio MM60","Precio MB51",
            'Costo Estándar', 'Costo Esperado',"Costo Tomado",
           'Variación Consumo', 'Variación Precio',"Control de precios"]]
    
    temp=dfMb51[dfMb51["Clase de movimiento"]=="101"][["Orden","Centro","Material","Clase de movimiento"]].rename(columns={"Material":"Receta"})

    temp["Cab"]="Cab"

    if not dfComp.shape[0]==dfComp.merge(temp,on=["Orden","Centro","Receta","Clase de movimiento"],how="left").shape[0]:
        raise Exception("Cabeceras MB51 inserta filas")

    dfComp=dfComp.merge(temp,on=["Orden","Centro","Receta","Clase de movimiento"],how="left")
    
    for i in ["Costo Estándar","Costo Esperado","Costo Tomado","Variación Consumo","Variación Precio"]:
        dfComp[i].fillna(0,inplace=True)
    #aca1
    dfComp.loc[(dfComp["Receta"]!=dfComp["Material"])&(dfComp["Cab"]=="Cab"),"Cab"]="Consumos"
    
    del temp["Cab"]
    temp1=dfComp[dfComp["Cab"]=="Cab"][["Orden","Centro","Receta","Clase de movimiento"]].drop_duplicates()
    temp1["Cab"]="Cab"

    temp1=temp1.rename(columns={"Receta":"Material"})

    if not dfMb51.shape[0]==dfMb51.merge(temp1,on=["Orden","Centro","Material","Clase de movimiento"],how="left").shape[0]:
        raise Exception("Temp1 añade filas")

    dfMb51=dfMb51.merge(temp1,on=["Orden","Centro","Material","Clase de movimiento"],how="left")

    temp=dfMb51[dfMb51["Cab"]!="Cab"]
    del temp["Cab"]
    del temp["Precio MB51"]

    temp=temp[temp["Clase de movimiento"]=="101"]

    temp=temp.rename(columns={"Material":"Receta","Ctd.en UM entrada":"Cantidad tomada (EINHEIT)","Importe ML":"Valor de la toma (WAERS)"})

    dfCab=pd.read_excel(ruta+"\{}\Consumos\{}. Cabeceras de orden.xlsx".format(tiempo[0],tiempo[1]),usecols=colsCab,converters=convCab)
    dfCab=dfCab.rename(columns={"Cantidad orden (GMEIN)":"Cantidad Plan PT","Cantidad entregada (GMEIN)":"Cantidad Real PT",
                                   'Número material':"Receta",'Texto breve material':"Desc. Receta",
                               'Unidad de medida (=GMEIN)':'Unidad medida base (=EINHEIT)'})

    dfCab["Status"]=dfCab["Status de sistema"].str[:4]

    cols=['Centro', 'Orden', 'Receta', 'Desc. Receta','Cantidad Plan PT',
           'Cantidad Real PT', 'División', 'Fecha liberac.real', 'Status','Unidad medida base (=EINHEIT)']

    if not temp.shape[0]==temp.merge(dfCab[cols],on=['Centro', 'Orden', 'Receta'],
              how="left").shape[0]:
        raise Exception("Cabeceras inserta filas")

    temp=temp.merge(dfCab[cols],on=['Centro', 'Orden', 'Receta'],how="left")

    temp["División"].fillna("No encontrado",inplace=True)

    temp=temp[~temp["Status"].isna()]

    temp["Cab"]="Cab"

    temp["Material"]=temp["Receta"].copy()
    temp["Texto breve material"]=temp["Desc. Receta"].copy()

    temp["Almacén"]="No encontrado"

    temp["Cantidad Plan MP"]=temp["Cantidad Plan PT"].copy()
    temp["División"]=temp["División"].fillna("No encontrado")

    for i in ['Precio tomado', 'Precio esperado', 'Precio Plan', 'Precio MM60','Precio MB51']:
        temp[i]=0

    for i in ['Costo Estándar', 'Costo Esperado', 'Costo Tomado']:
        temp[i]=temp['Valor de la toma (WAERS)'].copy()

    temp['Variación Consumo']=0
    temp['Variación Precio']=0

    dfComp.reset_index(inplace=True, drop=True)
    temp.reset_index(inplace=True, drop=True)

    dfComp["Cab"].fillna("Consumos",inplace=True)
    temp["Cantidad necesaria (EINHEIT)"]=temp["Cantidad tomada (EINHEIT)"]
    
    for i in temp.columns:
        if "Precio" not in i:
            if (temp[i].dtype=="float64") or (temp[i].dtype=="int64"):
                temp[i]=-temp[i]
    
    dfComp=pd.concat([dfComp,temp])

    dfEj=pd.read_excel(ruta+"\{}\Ejecución\{}. Cuenta 7 Industria.xlsx".format(tiempo[0],tiempo[1]),usecols=colsEjec,converters=convEjec)

    dfEj=dfEj[dfEj["Clase de coste"].isin(["PP1001","PP1002"])]

    dfEj.loc[:,"Valor de la toma (WAERS)"]=-dfEj["Valor/mon.inf."]
    dfEj.loc[:,"Variación Precio"]=-dfEj["Desv.precio fija"]
    dfEj.loc[:,"Cantidad necesaria (EINHEIT)"]=-dfEj["Cantidad total"]

    del dfEj["Valor/mon.inf."]
    del dfEj["Desv.precio fija"]
    del dfEj["Cantidad total"]

    dfEj=dfEj.groupby(["Clase de coste","Denom.clase de coste","Orden partner"]).sum().reset_index()

    dfEj=dfEj.rename(columns={"Clase de coste":"Material","Denom.clase de coste":"Texto breve material","Orden partner":"Orden"})

    if not dfEj.shape[0]==dfEj.merge(dfCab,on=["Orden"],how="left").shape[0]:
        raise Exception("Cab inserta filas")

    dfEj=dfEj.merge(dfCab,on=["Orden"],how="left")

    dfEj["Unidad medida base (=EINHEIT)"]="H"

    dfEj["Cantidad tomada (EINHEIT)"]=dfEj["Cantidad necesaria (EINHEIT)"]
    dfEj["Cantidad Plan MP"]=dfEj["Cantidad necesaria (EINHEIT)"]

    dfEj["Almacén"]="No encontrado"

    dfEj['Precio tomado']=dfEj['Valor de la toma (WAERS)'].divide(dfEj['Cantidad necesaria (EINHEIT)'],fill_value=0).fillna(0)

    for i in ['Precio esperado', 'Precio Plan', 'Precio MM60','Precio MB51']:
        dfEj[i]=dfEj['Precio tomado']

    dfEj['Costo Estándar']=dfEj['Valor de la toma (WAERS)']-dfEj['Variación Precio']
    dfEj['Costo Esperado']=dfEj['Costo Estándar']
    dfEj['Costo Tomado']=dfEj['Valor de la toma (WAERS)']
    dfEj['Variación Consumo']=0

    for i in dfEj.columns:
        if i not in dfComp.columns:
            del dfEj[i]

    dfEj["Cab"]="Conversión"
    dfEj["Clase de movimiento"]="CONV"
    #break
    dfEj["Var Cuenta 7"]=dfEj["Variación Precio"]
    dfEj["Factor"]=dfEj['Cantidad Real PT'].divide(dfEj['Cantidad Plan PT'],fill_value=0).fillna(0)
    dfEj["Variación Consumo"]=(dfEj["Valor de la toma (WAERS)"]-dfEj["Variación Precio"])*(1-dfEj["Factor"])
    del dfEj["Factor"]
    
    dfComp["Var Cuenta 7"]=0
    
    dfComp=pd.concat([dfComp,dfEj])

    dfComp=dfComp.rename(columns={"Cab":"Clase"})
    
    vT=dfComp[["Orden","Costo Tomado","Variación Consumo","Variación Precio","Receta"]].copy()
    vT=vT.groupby(["Orden","Receta"]).sum().reset_index()

    vT=vT.rename(columns={"Costo Tomado":"Variación Total"})

    vT["Otras Variaciones"]=vT["Variación Total"]-vT["Variación Consumo"]-vT["Variación Precio"]

    vT["Clase"]="Cab"

    if not dfComp.shape[0]==dfComp.merge(vT[["Orden","Receta","Clase","Variación Total","Otras Variaciones"]],how="left",on=["Orden","Receta","Clase"]).shape[0]:
        raise Exception("vT inserta filas")

    dfComp=dfComp.merge(vT[["Orden","Receta","Clase","Variación Total","Otras Variaciones"]],how="left",on=["Orden","Receta","Clase"])
    dfComp["Variación Total"].fillna(0,inplace=True)
    dfComp["Otras Variaciones"].fillna(0,inplace=True)
    dfComp["Clase"]=dfComp["Clase"].replace("Cab","Cabecera")
    dfComp=dfComp.rename(columns={"Variación Total":"Neto Orden"})
    
    dfCentro=pd.read_excel(rutaM+"\Centros.xlsx",usecols=["Centro","Descripción Centro"],converters={"Centro":str})

    dfComp=dfComp.merge(dfCentro,how="left",on=["Centro"])
    
    
    dfM=dfM[["Material","Centro","Control de precios"]].rename(columns={"Material":"Receta",
                                                                        "Control de precios":"Control de precios Receta"})

    if not dfComp.shape[0]==dfComp.merge(dfM,on=["Receta","Centro"],how="left").shape[0]:
        raise Exeption("MM60 inserta filas")

    dfComp=dfComp.merge(dfM,on=["Receta","Centro"],how="left")
    
    for i in dfComp.columns:
        if dfComp[i].dtype == "object":
            dfComp[i].fillna("No encontrado",inplace=True)
        if (dfComp[i].dtype == "float64") or (dfComp[i].dtype == "int64"):
            dfComp[i].fillna(0.0,inplace=True)
        if (dfComp[i].dtype == "datetime64[ns]") or (dfComp[i].dtype == "int64"):
            dfComp[i].fillna(datetime.datetime(tiempo[0],tiempo[1],1),inplace=True)
    
    cols=['Orden', 'Centro', 'Descripción Centro', 'Receta', 'Desc. Receta', 'Clase', 'Clase de movimiento',
       'Material', 'Texto breve material', 'Unidad medida base (=EINHEIT)',
       'Cantidad necesaria (EINHEIT)', 'Cantidad tomada (EINHEIT)',
       'Valor de la toma (WAERS)', 'Almacén', 'División', 'Fecha liberac.real',
       'Status', 'Cantidad Plan PT', 'Cantidad Real PT', 'Cantidad Plan MP',
       'Precio tomado', 'Precio esperado', 'Precio Plan', 'Precio MM60',
       'Precio MB51', 'Costo Estándar', 'Costo Esperado', 'Costo Tomado',
       'Variación Consumo', 'Variación Precio',  'Otras Variaciones','Neto Orden',"Var Cuenta 7",
         'Control de precios','Control de precios Receta']

    dfComp=dfComp[cols]
    dfComp.loc[(dfComp["Clase"]=="Consumos") & (dfComp["Clase de movimiento"].isin(["101","102"])),"Clase"] = "Otras Entregas"
    dfComp.loc[(dfComp["Clase"]=="Consumos") & (dfComp["Clase de movimiento"].isin(["531","532"])),"Clase"] = "Subproductos"
    
    dfComp.to_excel(rutaR+"\Consumos\{}\{}. Consumos.xlsx".format(tiempo[0],tiempo[1]),index=None)
    print(tiempo)

(2023, 1)
(2023, 2)
(2023, 3)
(2023, 4)
(2023, 5)
